In [1]:
import cv2
import math
import scipy
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

In [2]:
def get_non_uniform_sampling(source, steps):
    shape = np.shape(source)
    height = shape[0]
    length = shape[1]
    
    x_centre = int(length/2)
    y_centre = int(height/2)
    radius = int(min(length, height)/2)
    
    pi = np.pi
    multiplier = 0.5
    x_coordinates = []
    y_coordinates = []

    for r in steps:
        number_of_points = (1/multiplier) * 2

        for j in range(int(number_of_points)):
            theta = j*pi*multiplier
            x = x_centre + (r * np.cos(theta))
            y = y_centre + (r * np.sin(theta))
            
            if(x<length and x>0 and y<height and y>0):
                x_coordinates.append(int(x))
                y_coordinates.append(int(y))

        multiplier = multiplier/2
    
    return x_coordinates, y_coordinates
    
    return x_coordinates, y_coordinates

def get_roi_values(src, x , y, window_size):
    height = np.shape(img)[0]
    length = np.shape(img)[1]
    
    left = x - int(window_size/2)
    left = max(0, left)
    
    right = x + int(window_size/2)
    right = min(length, right)
    
    top = y - int(window_size/2)
    top = max(0, top)
    
    bottom = y + int(window_size/2)
    bottom = min(height, bottom)
    
    slice_arr = src[top:bottom, left:right]
    
    return slice_arr, np.mean(slice_arr), np.std(slice_arr)

def get_mean_std(src, x_coordinates, y_coordinates, window_size):
    mean = []
    std = []
    number_of_points = np.shape(x_coordinates)[0]
    
    for i in range(number_of_points):
        slice_arr, slice_mean, slice_std = get_roi_values(src, x_coordinates[i], y_coordinates[i], window_size)
        mean.append(slice_mean)
        std.append(slice_std)
    
    return mean, std

def sort_counterclockwise(points, centre = None):
    if centre:
        centre_x, centre_y = centre
    else:
        centre_x, centre_y = sum([x for x,_ in points])/len(points), sum([y for _,y in points])/len(points)
    angles = [math.atan2(y - centre_y, x - centre_x) for x,y in points]
    counterclockwise_indices = sorted(range(len(points)), key=lambda i: angles[i])
    counterclockwise_points = [points[i] for i in counterclockwise_indices]
    return counterclockwise_points

def get_non_uniform_sampling_final(img, number_of_steps):
    shape = np.shape(img)
    height = shape[0]
    length = shape[1]
    radius = int(min(length, height)/2)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    edged = cv2.Canny(blurred, 10, 100)
    arr_edge = np.where(edged==255)
    x_edge = arr_edge[1]
    y_edge = arr_edge[0]
    
    steps = np.linspace(0, radius, number_of_steps)

    for i in range(number_of_steps):
        steps[i] = steps[i] - int(math.pow(i, 3)) + int(math.pow(i, 1.5))

    steps = np.array(steps, dtype=int)
    
    x_coordinates, y_coordinates = get_non_uniform_sampling(img, steps)
    points = []
    for i , j in zip(x_edge, y_edge):
        points.append((i,j))
        
    sorted_points = sort_counterclockwise(points)
    t = np.transpose(sorted_points)
    x_edge = t[0]
    y_edge = t[1]

    interval = 2**(number_of_steps + 1)

    x_edge = np.array(x_edge[0::interval] ,dtype=int)
    y_edge = np.array(y_edge[0::interval], dtype=int)
    
    x_coordinates = np.concatenate([x_coordinates, x_edge])
    y_coordinates = np.concatenate([y_coordinates, y_edge])
    
    return x_coordinates, y_coordinates

def enhance_image(img, image_window_size, background_window_size, threshold):
    (B,G,R) = cv2.split(img)
    height = np.shape(G)[0]
    length = np.shape(G)[1]
    
    # Non Uniform Sampling --------------------------------------------------------------------------------------
    x_coordinates, y_coordinates = get_non_uniform_sampling_final(img, 5)
    
    # Extrapolated mean and standard deviation ------------------------------------------------------------------
    mean, std = get_mean_std(G, x_coordinates, y_coordinates, image_window_size)
    
    xi = np.arange(0, length, 1)
    yi = np.arange(0, height, 1)
    x_grid, y_grid = np.meshgrid(xi, yi)

    points = []
    for i , j in zip(x_coordinates, y_coordinates):
        points.append([i,j])
    
    interpolated_mean = scipy.interpolate.griddata(points, mean, (x_grid, y_grid), method='linear', fill_value=0, rescale=False)
    interpolated_std = scipy.interpolate.griddata(points, std, (x_grid, y_grid), method='linear', fill_value=0, rescale=False)
    
    # Mahalanobis Distance---------------------------------------------------------------------------------------
    mahalanobis = np.empty(np.shape(G))
    background = np.zeros(np.shape(G))
    G = np.array(G)
    
    interpolated_mean = np.array(interpolated_mean)
    interpolated_std = np.array(interpolated_std)
    mahalanobis = np.abs((G - interpolated_mean)/interpolated_std)
    
    indexes = np.where(mahalanobis <= threshold)
    background = np.zeros([height, length], dtype=int)
    background[indexes] = 1
    mult = background*G
    
    # Background operations---------------------------------------------------------------------------------------
    mean_mult = []
    std_mult = []

    for i in range(length):
        for j in range(height):
            mult_slice, temp_mean, temp_std = get_roi_values(mult, i , j, background_window_size)
            mult_slice = mult_slice[(mult_slice != 0)]
            if(not np.any(mult_slice)):
                mean_mult.append(0) 
                std_mult.append(0)
            else:
                mean_m = np.mean(mult_slice)
                std_m = np.std(mult_slice)
                mean_mult.append(mean_m)
                std_mult.append(std_m)
    
    U = np.empty(np.shape(G))
    SM = np.array(mean_mult)
    SA = np.array(std_mult)
    SM = SM.reshape(height, length,  order='F')
    SA = SA.reshape(height, length, order='F')
    U = (G - SM) / SA
    
    U[np.isnan(U)] = 0
    U[~np.isfinite(U)] = 0
    U_norm = (U - np.min(U))/(np.max(U)- np.min(U))
    
    # Final Processing ------------------------------------------------------------------------------------------
    v = np.empty(np.shape(G))
    R = np.array(R)
    G = np.array(G)
    B = np.array(B)

    v = np.maximum(np.maximum(R,G),B)
    R_corr = (U_norm/v)*R
    G_corr = (U_norm/v)*G
    B_corr = (U_norm/v)*B

    R_corr[np.isnan(R_corr)] = 0
    G_corr[np.isnan(G_corr)] = 0
    B_corr[np.isnan(B_corr)] = 0
    
    enhanced_image = cv2.merge([B_corr, G_corr, R_corr])

    return enhanced_image

In [3]:
img = cv2.imread("./Images/OD0058EY.JPG")
img[1200:1248, 0:100, :] = 0

(B,G,R) = cv2.split(img)

print("Image Dimensions -", np.shape(img))
# cv2.imshow("Source Image", G)
# cv2.waitKey(0)

Image Dimensions - (1248, 1664, 3)


In [ ]:
a = enhance_image(img, 50, 125, 0.9)

cv2.namedWindow('Enhanced Image', cv2.WINDOW_NORMAL)
cv2.imshow("Enhanced Image", a)
cv2.waitKey(0)

In [ ]:
# x_coordinates, y_coordinates = get_non_uniform_sampling_final(img, 7)
# number_of_points = (np.shape(x_coordinates))[0]
# print("The number of points is", number_of_points)

In [ ]:
# img[1200:1248, 0:100, :] = 0

In [ ]:
# plt.figure(figsize = (8,8))
# plt.imshow(G)
# plt.scatter(x_coordinates, y_coordinates, color='red')
# plt.colorbar()
# plt.title("Image and Sampling Points")

In [ ]:
# mean, std = get_mean_std(G, x_coordinates, y_coordinates, 50)

In [ ]:
# xi = np.arange(0, np.shape(G)[1], 1)
# yi = np.arange(0, np.shape(G)[0], 1)
# x_grid, y_grid = np.meshgrid(xi, yi)

# points = []
# for i , j in zip(x_coordinates, y_coordinates):
#     points.append([i,j])

In [ ]:
# interpolated_mean = scipy.interpolate.griddata(points, mean, (x_grid, y_grid), method='linear', fill_value=0, rescale=False)

# plt.figure(figsize = (8,8))
# plt.imshow(interpolated_mean)
# plt.colorbar()
# plt.title("Interpolated Mean")

In [ ]:
# interpolated_std = scipy.interpolate.griddata(points, std, (x_grid, y_grid), method='linear', fill_value=0, rescale=False)

# plt.figure(figsize = (8,8))
# plt.imshow(interpolated_std)
# plt.colorbar()
# plt.title("Interpolated Standard Deviation")

In [ ]:
# height = np.shape(G)[0]
# length = np.shape(G)[1]

# mahalanobis = np.empty(np.shape(G))
# background = np.zeros(np.shape(G))

# G = np.array(G)
# interpolated_mean = np.array(interpolated_mean)
# interpolated_std = np.array(interpolated_std)

# mahalanobis = np.abs((G - interpolated_mean)/interpolated_std)
# plt.figure(figsize = (8,8))
# plt.imshow(mahalanobis, cmap='binary')
# plt.colorbar()
# plt.title("Mahalanobis Distance")

In [ ]:
# threshold = 1
# indexes = np.where(mahalanobis <= threshold)
# background = np.zeros([height, length], dtype=int)
# background[indexes] = 1

# plt.figure(figsize = (8,8))
# plt.imshow(background, cmap="binary_r")
# plt.colorbar()
# plt.title("Background")

In [ ]:
# background = np.array(background)
# G = np.array(G)
# mult = background*G

# mult = mult.astype(np.uint8)
# plt.figure(figsize = (8,8))
# plt.imshow(mult)
# plt.colorbar()
# plt.title("Mult")

In [ ]:
# mean_mult = []
# std_mult = []
# background_window_size = 125
  
# for i in range(length):
#     for j in range(height):
#         mult_slice, temp_mean, temp_std = get_roi_values(mult, i , j, background_window_size)
#         mult_slice = mult_slice[(mult_slice != 0)]
#         if(not np.any(mult_slice)):
#             mean_mult.append(0) 
#             std_mult.append(0)
#         else:
#             mean_m = np.mean(mult_slice)
#             std_m = np.std(mult_slice)
#             mean_mult.append(mean_m)
#             std_mult.append(std_m)

In [ ]:
# U = np.empty(np.shape(G))
# SM = np.array(mean_mult)
# SA = np.array(std_mult)
# SM = SM.reshape(height, length,  order='F')
# SA = SA.reshape(height, length, order='F')
# U = (G - SM) / SA

In [ ]:
# U[np.isnan(U)] = 0
# U[~np.isfinite(U)] = 0
# U_norm = (U - np.min(U))/(np.max(U)- np.min(U))

# plt.figure(figsize = (8,8))
# plt.imshow(U_norm)
# plt.colorbar()
# plt.title("U norm")

In [ ]:
# plt.figure(figsize = (8,8))
# plt.imshow(SA)
# plt.colorbar()
# plt.title("SA")

# plt.figure(figsize = (8,8))
# plt.imshow(SM)
# plt.colorbar()
# plt.title("SM")

In [ ]:
# v = np.empty(np.shape(G))
# R = np.array(R)
# G = np.array(G)
# B = np.array(B)

# v = np.maximum(np.maximum(R,G),B)
# R_corr = (U_norm/v)*R
# G_corr = (U_norm/v)*G
# B_corr = (U_norm/v)*B

# R_corr[np.isnan(R_corr)] = 0
# G_corr[np.isnan(G_corr)] = 0
# B_corr[np.isnan(B_corr)] = 0

# plt.figure(figsize = (8,8))
# plt.imshow(R_corr)
# plt.colorbar()
# plt.title("Corrected R plane")

# plt.figure(figsize = (8,8))
# plt.imshow(G_corr)
# plt.colorbar()
# plt.title("Corrected G plane")

# plt.figure(figsize = (8,8))
# plt.imshow(B_corr)
# plt.colorbar()
# plt.title("Corrected B plane")

In [ ]:
# enhanced_image = cv2.merge([B_corr, G_corr, R_corr])

In [ ]:
# cv2.namedWindow('Enhanced Image', cv2.WINDOW_NORMAL)
# cv2.imshow("Enhanced Image", enhanced_image)
# cv2.waitKey(0)